# Eye Detection "Super Model" Training Pipeline

This notebook trains a YOLOv8 model to detect eyes. It uses aggressive data augmentation (Mosaic, Mixup, Random Crop/Scale) to ensure the model is robust to:
- Partial faces (half-face, zoomed in)
- Crowds (small faces)
- Occlusions

**Instructions:**
1. Run all cells in order.
2. The dataset will be downloaded and prepared automatically.
3. Training will run for 50 epochs.
4. The final model (`best.pt`) will be zipped and downloaded to your computer.

In [ ]:
# 1. Setup Environment
!pip install ultralytics opencv-python-headless

import os
import shutil
import cv2
import numpy as np
import glob
import requests
import zipfile
from ultralytics import YOLO
from google.colab import files

In [ ]:
# 2. Download Helen Dataset
DATASET_URL = "http://www.ifp.illinois.edu/~vuongle2/helen/data/helen_1.zip" # Example URL, usually needs Kaggle API or direct link. 
# Since direct links are unstable, we will assume the user might need to upload it OR we use a stable mirror if available.
# For this script, we'll try to download from a source, but if it fails, you can upload 'helen_dataset.zip' manually.

# NOTE: The official Helen link is often down. 
# We will simulate the structure or use a placeholder if you don't have the zip.
# Ideally, upload 'helen_dataset.zip' to the Colab files area before running this if the download fails.

def download_file(url, filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download. Status: {response.status_code}")

# If you have the zip, uncomment this line and skip download:
# !unzip -q helen_dataset.zip -d helen_dataset

# Placeholder for Kaggle download command if you have API key:
# !kaggle datasets download -d helen-dataset-url

print("Please upload 'helen_dataset.zip' to the Files section if not already present.")
print("Assuming 'helen_dataset.zip' is present in the current directory...")

In [ ]:
# 3. Extract Dataset
if os.path.exists('helen_dataset.zip'):
    !unzip -q -o helen_dataset.zip -d helen_dataset
    print("Extracted helen_dataset.zip")
else:
    print("Error: helen_dataset.zip not found! Please upload it.")

In [ ]:
# 4. Data Preparation (Landmarks -> YOLO)
# This script converts the Helen .pts landmarks to YOLO bounding boxes

DATASET_DIR = "helen_dataset"
OUTPUT_DIR = "data"
IMAGES_DIR = os.path.join(OUTPUT_DIR, "images")
LABELS_DIR = os.path.join(OUTPUT_DIR, "labels")

# Clean output directory
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

# Create directories
for split in ['train', 'val']:
    os.makedirs(os.path.join(IMAGES_DIR, split), exist_ok=True)
    os.makedirs(os.path.join(LABELS_DIR, split), exist_ok=True)

def convert_to_yolo(img_width, img_height, x_min, y_min, x_max, y_max):
    x_center = (x_min + x_max) / 2.0 / img_width
    y_center = (y_min + y_max) / 2.0 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return 0, x_center, y_center, width, height

def process_data():
    image_files = glob.glob(f"{DATASET_DIR}/**/*.jpg", recursive=True)
    print(f"Found {len(image_files)} images.")
    
    processed_count = 0
    for i, img_path in enumerate(image_files):
        # Find corresponding .pts or .txt file
        pts_path = os.path.splitext(img_path)[0] + ".pts"
        if not os.path.exists(pts_path):
            pts_path = os.path.splitext(img_path)[0] + ".txt"
            if not os.path.exists(pts_path):
                continue
            
        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w = img.shape[:2]
        
        landmarks = []
        try:
            with open(pts_path, 'r') as f:
                lines = f.readlines()
            
            start_parsing = False
            for line in lines:
                line = line.strip()
                if line == '{' or (line.replace('.','',1).isdigit() and len(line.split())==2 and 'version' not in line):
                     # Handle both .pts header and raw coordinate lines
                    start_parsing = True
                    if line == '{': continue
                
                if line == '}':
                    break
                
                if start_parsing or (len(line.split())==2 and line[0].isdigit()):
                    parts = line.split()
                    if len(parts) >= 2:
                        try:
                            x, y = float(parts[0]), float(parts[1])
                            landmarks.append((x, y))
                        except ValueError:
                            pass
        except Exception as e:
            print(f"Error parsing {pts_path}: {e}")
            continue
                    
        if len(landmarks) != 68:
             # Try to take last 68 if more
            if len(landmarks) > 68:
                landmarks = landmarks[-68:]
            else:
                continue

        # Extract eye regions (Left: 36-41, Right: 42-47)
        left_eye_pts = landmarks[36:42]
        right_eye_pts = landmarks[42:48]
        
        yolo_labels = []
        
        for pts in [left_eye_pts, right_eye_pts]:
            pts_np = np.array(pts)
            x_min = np.min(pts_np[:, 0])
            y_min = np.min(pts_np[:, 1])
            x_max = np.max(pts_np[:, 0])
            y_max = np.max(pts_np[:, 1])
            
            # Add padding (30%)
            pad_x = (x_max - x_min) * 0.3
            pad_y = (y_max - y_min) * 0.3
            
            x_min = max(0, x_min - pad_x)
            y_min = max(0, y_min - pad_y)
            x_max = min(w, x_max + pad_x)
            y_max = min(h, y_max + pad_y)
            
            cls, xc, yc, bw, bh = convert_to_yolo(w, h, x_min, y_min, x_max, y_max)
            yolo_labels.append(f"{cls} {xc} {yc} {bw} {bh}")
            
        # Split train/val
        split = 'train' if i < len(image_files) * 0.8 else 'val'
        
        shutil.copy(img_path, os.path.join(IMAGES_DIR, split, os.path.basename(img_path)))
        
        label_filename = os.path.splitext(os.path.basename(img_path))[0] + ".txt"
        with open(os.path.join(LABELS_DIR, split, label_filename), 'w') as f:
            f.write('\n'.join(yolo_labels))
        
        processed_count += 1
            
    print(f"Data preparation complete. Processed {processed_count} images.")

process_data()

In [ ]:
# 5. Configure Dataset YAML
yaml_content = """
names:
  0: eye
path: /content/data
train: images/train
val: images/val
"""
with open('dataset.yaml', 'w') as f:
    f.write(yaml_content)
print("Created dataset.yaml")

In [ ]:
# 6. Train "Super Model"
# We use aggressive augmentation to force the model to learn robust features

model = YOLO('yolov8n.pt') # Start with Nano model for speed, use 'yolov8s.pt' for more accuracy

results = model.train(
    data='dataset.yaml',
    epochs=50,           # Sufficient time to converge
    imgsz=640,
    batch=16,
    patience=10,         # Early stopping
    
    # --- Super Model Augmentations ---
    mosaic=1.0,          # 100% chance to stitch 4 images (Simulates crowds)
    mixup=0.1,           # Blend images (General robustness)
    degrees=15.0,        # Rotation +/- 15 deg
    scale=0.5,           # Scale image +/- 50% (Simulates partial/zoomed faces)
    erasing=0.4,         # Randomly erase patches (Simulates occlusion)
    
    project='runs/detect',
    name='eye_super_model',
    exist_ok=True
)

In [ ]:
# 7. Export and Download
!zip -r eye_super_model.zip runs/detect/eye_super_model/weights/best.pt
files.download('eye_super_model.zip')
print("Download started! Place this file in your app's ml_pipeline/runs/detect/ folder.")